In [ ]:
# for auto-reloading rennet modules
%load_ext autoreload
%autoreload 1

# py2.7 compat
from __future__ import division, print_function
from six.moves import zip, range, zip_longest

In [ ]:
import os
import sys

rennet_data_root = os.path.join("..", "..", "data")
rennet_x_root = os.path.join("..", "..")

# Preparing `rennet_model` from trained `keras_model`

The goal of this notebook is to document how:
- the trained models were analysed `[SKIPPED]`
- two trained models were chosen to created a combined `rennet_model`
- the `rennet_model` reading and application class was implemented
- the `rennet_model` was tuned to perform on some data
- the `rennet_model` was exported to `model.h5` to be used by `annonet`

### Analyzing trained `keras_model`

The analysis was done of all the `keras_model` checkpoints saved for all the training configs during the training phase.

One `keras_model` checkpoint per config, and was __manually copied__ to a new file `model.h5` in the same activity output directory as the config.
- For Abdullah's research, __the last__ `keras_model` checkpoint (for the last epoch of the last pass) of each config was chosen as the `model.h5`.
- This was done because the training had stabilized, and there weren't many significant differences between the models' performance after the training had stabilized.
- The evaluations of each config were performed on the entire testing split.

---
## Combine `keras_model`s for `m-n/keepzero` and `m-n/skipzero`

- It was observed that __mean-normalization__ gave, overall, the best results for detecting double-talks, irrespective of the sub-sampling applied.
- It was observed that `m-n/keepzero` config detected all three classes, very good with silence, very precise with double-talks (but with very low recall).
- It was observed that `m-n/skipzero-20one` detected a lot more double-talks, but at loss of precision, and, as expected, without ever predicting for silence.

Based on these observations, the decision was made to combine the best models from the two configs below in parallel to form the final `rennet_model`:
1. `m-n/keepzero`
2. `m-n/skipzero-20one`

The combined model will take two inputs, and produce to softmax-predictions for the respectively ordered models.
The inputs to both the models are the same, hence the feature extraction will be the same for both the inputs.

The two ouputs of the model will be __merged (and normalized)__ based on some strategy to get new _faux_-softmax posteriors. 
Viterbi smoothing will then be applied to produce the final predictions.

It was decided to first combine and export the two chosen `keras_model`s, and implement an ___exclusive___ `rennet_model` class in `rennet.models` which will be responsible for reading this combined model and applying it to a given speech file.

The class includes the decision made on how to merge the predictions from the two models before viterbi smoothing.

The merging strategy used is that of taking a weighted average of the two predictions, the weights being different for each of the three classes. Other strategies were investigated, but this one gave the most consistent result.

The merging weights, in addition to the window size used for normalization (all parameters of the `rennet_model` are initialized in it's `__init__` method) are tunable.
They have been set to hard-coded values based on which perform best on KA3 (client) data overall.

Most of the ther hard-coded parameters are specific and exclusive to this particular combined model.

In [ ]:
from h5py import File as hf
from keras.models import load_model

In [ ]:
# Choose the models to merge

outputs = os.path.join(rennet_x_root, "outputs", "fisher-sample")
keepzero = os.path.join(outputs, "m-n/keepzero/model.h5")  # chosen model
skipzero = os.path.join(outputs, "m-n/skipzero/model.h5")  # chosen model

assert os.path.exists(keepzero)
assert os.path.exists(skipzero)

In [ ]:
# Load the chosen models
m1 = load_model(keepzero)
m2 = load_model(skipzero)

In [ ]:
m1.summary()

In [ ]:
m2.summary()

In [ ]:
%aimport rennet.utils.keras_utils
import rennet.utils.keras_utils as ku

In [ ]:
# combine the models in parallel - ORDER MATTERS!!
mm = ku.combine_keras_models_parallel([m1, m2])
mm.summary()

In [ ]:
# combined the combined model
mm.compile(
    optimizer='adamax',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'],
)

In [ ]:
# export the combined `keras_model`
model_export_path = os.path.abspath(os.path.join(rennet_data_root, "models", "model.h5"))

mm.save(model_export_path, overwrite=False)

# We will add stuff to this model to make it into a compatible `rennet_model`

In [ ]:
# read the hdf5 file
f = hf(model_export_path, 'a')

print(list(f.keys()))  # what keras has exported

In [ ]:
%aimport rennet.utils.py_utils
import rennet.utils.py_utils as pu

import glob

In [ ]:
# collect the raw Viterbi priors from the trn.h5 and val.h5
pickles_root = os.path.join(rennet_data_root, "working", "fisher", "fe_03_p1", "wav-8k-mono", "pickles")

pickles_dir = sorted(glob.glob(os.path.join(pickles_root, "*")))  # all featx pickles directories
print("Pickles directories found:\n", "\n".join(str(p) for p in pickles_dir), '\n')


# ATTENTION: make sure this is the same one on which the models chosen above were trained on

pickles_dir = pickles_dir[-1]  # choose the latest one
print("Pickles directory CHOSEN:\n", pickles_dir)

val_h5 = os.path.join(pickles_dir, "val.h5")
trn_h5 = os.path.join(pickles_dir, "trn.h5")

with hf(val_h5, 'r') as f_a:
    vinit = f_a["viterbi/init"][()]
    vtran = f_a["viterbi/tran"][()]
    vpriors = f_a["viterbi/priors"][()]

with hf(trn_h5, 'r') as f_a:
    tinit = f_a["viterbi/init"][()]
    ttran = f_a["viterbi/tran"][()]
    tpriors = f_a["viterbi/priors"][()]

init = vinit + tinit
tran = vtran + ttran#mgref. priors is undefined in the following line. assuming:
priors = vpriors + tpriors

In [ ]:
# save the viterbi priors in the combined model's h5 at `model/viterbi/...`
f.create_group('rennet/model')
f.create_group('rennet/model/viterbi')

f.create_dataset('rennet/model/viterbi/init', data=init)
f.create_dataset('rennet/model/viterbi/tran', data=tran)
f.create_dataset('rennet/model/viterbi/priors', data=priors)
f.flush()

In [ ]:
from rennet import __version__ as v
print(v)  # the current version of rennet

In [ ]:
# add the source version and the minimum supported version of rennet for which
# the model we are creating will be valid
f['rennet'].attrs['version_src'] = v
f['rennet'].attrs['version_min'] = "0.1.0"  # min-version when the rennet_model class was implemented
f.flush()

In [ ]:
%aimport rennet.models
import rennet.models as rm

In [ ]:
# Choose the rennet_model class which was implemented EXCLUSIVELY for this type of model

rennet_model = rm.DT_2_nosub_0zero20one_mono_mn

In [ ]:
model_name = str(rennet_model.__name__)

# ATTENTION: Make sure that the name of the unique class from `rennet/models.py` is the correct one for this model
print(model_name)  # DT_2_nosub_0zero20one_mono_mn

In [ ]:
# Add info about which model class `annonet` should use to read/apply this model
# Make sure this is the right class. It should have already been implemented

f['rennet/model'].attrs['name'] = model_name
f.flush()

In [ ]:
# Save everything and close
f.close()

# The model.h5 is now a `rennet_model`

print("The rennet_model was exported at:\n", model_export_path)

In [ ]:
# f.create_dataset('rennet/model/norm_winsec', data=100)

### IMPORTANT REMINDER

The the model that has been exported __will not be__ tracked by git.

Provide it to your respective user, either separately, or bundled in the zip of the package at the location (in the `rennet` package you are preparing) `data/models/model.h5`.

That location will be looked in by `annonet.py` to get the `rennet_model` and, based on the additional information that we added above, especially `rennet/model[name]`, the appropriate model class will be used to read and apply the model.

Make sure to test an independent copy of the package (without any environment variables set), especially by running `annonet.sh`, before sending to a user.

---
***

## Analysis `[REFERENCE]`

> The analysis code in the rest of the notebook is here only for your reference.
> 
> It is __not guaranteed to work as__, but I hope that you will be able to infer what's going on, and adapt it to your needs
>
> As a hint, this code was written was to analyze a KA3 file for which annotations were available, and the final exported elan file consists of both the true and predicted labels 

In [ ]:
import numpy as np
import pympi as pm
import matplotlib.pyplot as plt
plt.style.use('seaborn-muted')

from itertools import chain, repeat, groupby
from collections import OrderedDict
from copy import deepcopy

In [ ]:
# %% reloadable imports of `rennet` modules
import rennet.utils.audio_utils as au
import rennet.utils.label_utils as lu
import rennet.utils.np_utils as nu
import rennet.datasets.ka3 as k3
import rennet.utils.plotting_utils as pu
import rennet.utils.keras_utils as ku
from rennet import models as m

In [ ]:
# %% filepaths
working_raw_ka3 = os.path.join(rennet_data_root, "working", "ka3", "deutsch-01", "raw")
audiofp = os.path.join(working_raw_ka3, "media/DEU_pear_Iuna.wav")
labelfp = os.path.join(working_raw_ka3, "labels/DEU_pear_Iuna.xml")

assert os.path.exists(audiofp)
assert os.path.exists(labelfp)

In [ ]:
# models
modelfp = model_export_path

In [ ]:


# %%
model = m.DT_2_nosub_0zero20one_mono_mn(modelfp)


In [ ]:

# %%
with hf("./outputs/DEU_pear_Iuna.h5") as f:
    sad = f['sad'][()]
    dtd = f['dtd'][()]

In [ ]:

# %%
d = model.preprocess(audiofp)


In [ ]:


# %%
sad, dtd = model.predict(d)


In [ ]:


# %%
with hf("./outputs/DEU_pear_Iuna.h5") as f:
    f.create_dataset('sad', data=sad)
    f.create_dataset('dtd', data=dtd)


In [ ]:


# %%
label = k3.ActiveSpeakers.from_file(labelfp)


In [ ]:
# %%
nsamples = model.loadaudio(audiofp).shape[0]  # pylint: disable=no-member


In [ ]:
# %%
ends = lu.samples_for_labelsat(nsamples, model.hop_len, model.win_len)[10:-10]
Y = label.labels_at(ends, model.samplerate).sum(axis=1)
list(map(len, (sad, dtd, Y)))


In [ ]:
# %%
def p(true, pred, nclasses=3, onlydiag=True):
    nu.print_prec_rec(
        *nu.normalize_confusion_matrix(
            nu.confusion_matrix(true, pred, nclasses=nclasses)),
        onlydiag=onlydiag)



In [ ]:
# %%
p(Y, sad.argmax(axis=1))



In [ ]:
# %%
p(Y, dtd.argmax(axis=1))


In [ ]:
# %%
# we can even change the mergepreds_fn to something else
merged = model.mergepreds_fn([sad, dtd])
merged.shape



In [ ]:
# %%
p(Y, merged.argmax(axis=1))



In [ ]:
# %%
vit = lambda pred: lu.viterbi_smoothing(pred, model.vinit, model.vtran)



In [ ]:
# %%
p(Y, vit(sad))


In [ ]:
# %%
p(Y, vit(dtd))


In [ ]:
# %%
p(Y, vit(merged))



In [ ]:
# %%
seq = lu.ContiguousSequenceLabels.from_dense_labels(
    vit(merged),
    keep=model.seq_keep,
    min_start=model.seq_minstart,
    samplerate=model.seq_samplerate)



In [ ]:
# %%
tofile = "./outputs/{}.out.rrcomp.eaf".format(os.path.basename(audiofp))
tofile



In [ ]:
# %%
eaf = seq.to_eaf(
    to_filepath=tofile,
    linked_media_filepath=audiofp,
    annotinfo_fn=model.seq_annotinfo_fn)



In [ ]:
# %%
label_c = deepcopy(label)



In [ ]:
# %%
label_c.labels = label_c.labels.sum(axis=1)



In [ ]:
# %%
with label.samplerate_as(1000):
    print(label[:3])



In [ ]:
# %%
with label_c.samplerate_as(1000):
    print(label_c[:3])



In [ ]:
# %%
label_tiers = {
    0: "true_none",
    1: "true_single",
    2: "true_multiple",
}
annotinfo_fn = lambda label: lu.EafAnnotationInfo(tier_name=label_tiers[label])
label_c.to_eaf(
    to_filepath=tofile,
    eafobj=eaf,
    annotinfo_fn=annotinfo_fn,
)
